In [1]:
#!pip install ipykernel
import duckdb
import ibis
import polars as pl
from openai import OpenAI
import json
import pandas as pd

In [3]:
# Connect to your DuckDB database using Ibis
con = ibis.duckdb.connect('samhsa_data.db')
dbcon = duckdb.connect('samhsa_data.db')
con.list_tables() # List the tables in the db")

['Demographics',
 'EmploymentDetails',
 'LegalInfo',
 'SubstanceUseHistory',
 'TreatmentInformation',
 'teds_a_raw_2015_2019']

#### Data Ingestion : Creating Polars DataFrame 

We could use two apporaches here. USe a simple ibis connection to excute the select query and turn the resulting pandas frame into polars. 
OR use duckdb connection and arrow to input directly into polars without a panda - polar connversion. Approach 2  appears to be marginally faster for this amount of data. So I would be going ahead with that for the future tables. 


In [12]:
# Ibis method
demographics_pd = con.table('Demographics').execute() #you can add limit(100) before excecute to sample this data
# And then convert to polars 
demo_pl = pl.from_pandas(demographics_pd)


In [ ]:
demo_pl.head(2)
#demo_pl.columns

CASEID,ADMYR,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,PREG,VET,STFIPS,CBSA2010,REGION,DIVISION
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
20151524993,2015,6,2,5,3,4,2,1,2,2,-9,4,9
20151449532,2015,7,2,1,4,2,3,2,2,2,-9,4,9


In [13]:
# DuckDB Method to Load database table 
dbcon = duckdb.connect('samhsa_data.db')
pl_df = pl.from_arrow(dbcon.execute("SELECT * FROM Demographics").arrow())
dbcon.close()

In [21]:
pl_df.head(2)
#pl_df.columns

CASEID,ADMYR,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,PREG,VET,STFIPS,CBSA2010,REGION,DIVISION
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
20151524993,2015,6,2,5,3,4,2,1,2,2,-9,4,9
20151449532,2015,7,2,1,4,2,3,2,2,2,-9,4,9


In [8]:
print(df.shape)
df.head()

(100, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,20151524993,2015,6,2,5,3,4,2,1,2,2,-9,4,9
1,20151449532,2015,7,2,1,4,2,3,2,2,2,-9,4,9
2,20151431263,2015,5,2,9,4,1,-9,2,2,2,-9,4,9
3,20151401765,2015,9,2,1,-9,2,5,2,2,2,-9,4,9
4,20151461771,2015,6,2,8,4,1,3,2,2,2,-9,4,9


In [12]:
# Store your credentials in a json locally | Don't forget to add this file to your gitignore

credentials_path = 'credentials.json'

# Read Credential
with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']

# Pass this key to configure OpenAI's API client
client = OpenAI(api_key=openai_api_key)



In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)